# Using the segmented mirror in HCIPy

Just using the segmented mirror this time directly from the PASTIS module.

The `SegmentedMirror()` works with any segmented aperture from `HCIPy`, provided you put it in a form where the aperture function also returns the segment positions. Currently I only have that for the HiCAT/ATLAST aperture, but it's not hard to make once you have an aperture functions, since the only thing is literally to add the segment positions as a return.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import astropy.units as u
from hcipy import *

os.chdir('../../pastis/')
from config import CONFIG_PASTIS
import util as util

<span style="color:red"> *** Careful: *** </span>

**The segmented mirror in this notebook is based specifically on the classes and funcitons in the custom module `atlast_imaging`, which is deprecated. I have switched to using the `SegmentedDeformableMirror` from hcipy, however if you run this notebook as is, it should still work fine. Please see the up-to-date notebooks in the main directories for the current versions.**

**You should also be on the PASTIS commit `caf29ad`.**

In [ ]:
import atlast_imaging as atim

### Instantiate an SM

In [ ]:
# Parameters
which_tel = CONFIG_PASTIS.get('telescope', 'name')
pupil_size = CONFIG_PASTIS.getint('numerical', 'tel_size_px')
PUP_DIAMETER = CONFIG_PASTIS.getfloat(which_tel, 'diameter')

npix = pupil_size
wvln_hc = 638e-9
lamD = 15
samp = 4
diam = PUP_DIAMETER
norm = False

In [ ]:
# HCIPy grids and propagator
pupil_grid = make_pupil_grid(dims=1024, diameter=diam)
focal_grid = make_focal_grid(pupil_grid, samp, lamD, wavelength=wvln_hc)
prop = FraunhoferPropagator(pupil_grid, focal_grid)

In [ ]:
# Generate an ATLAST pupil
aper, seg_pos = atim.get_atlast_aperture(normalized=norm)
aper = evaluate_supersampled(aper, pupil_grid, 2)

imshow_field(aper)
plt.title('Aperture')

In [ ]:
# Instantiate the segmented mirror
sm = atim.SegmentedMirror(aper, seg_pos)

In [ ]:
# Make a pupil plane wavefront from aperture
wf = Wavefront(aper, wavelength=wvln_hc)
imshow_field(wf.intensity)
plt.title('Wavefront intensity')

### Field at SM pupil plane

In [ ]:
# Propagate the wf through the SM and display its intensity
# and phase in that pupil plane, before propagation to image plane.
wf_at_sm = sm(wf)

plt.figure(figsize=(18, 9))
plt.suptitle('WF prop through SM, still in pupil plane')

plt.subplot(1, 2, 1)
imshow_field(wf_at_sm.intensity, mask=aper)
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(wf_at_sm.phase, mask=aper, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

### Field in image plane after SM

In [ ]:
# Propagate from SM to image plane and display intensity and phase
im_ref = prop(sm(wf))   # or im_ref = prop(wf_at_sm)

plt.figure(figsize=(18, 9))
plt.suptitle('Image plane after SM')

plt.subplot(1, 2, 1)
imshow_field(np.log10(im_ref.intensity))
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(im_ref.phase, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

### Aberrate some segments in tip and tilt

In [ ]:
sm.flatten()
for i in [2, 34, 3, 19, 23, 29, 15]:
    sm.set_segment(i, 0, 10e-9, 10e-9)
for i in [27, 5, 17, 24, 21]:
    sm.set_segment(i, 0, 20e-9, 0)
for i in [18, 31, 11, 13, 20]:
    sm.set_segment(i, 0, 0, 30e-9)
print(sm.coef)

In [ ]:
# SM in pupil plane
wf_at_sm = sm(wf)

plt.figure(figsize=(18, 9))
plt.suptitle('SM in pupil plane with tilts')

plt.subplot(1, 2, 1)
imshow_field(wf_at_sm.intensity, mask=aper)
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(wf_at_sm.phase, mask=aper, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

In [ ]:
# Image after SM with tilts
im = prop(sm(wf))

plt.figure(figsize=(18, 9))
plt.suptitle('Image plane after SM with tilts')

plt.subplot(1, 2, 1)
imshow_field(np.log10(im.intensity))
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(im.phase, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

### Flatten the SM and apply pistons instead

In [ ]:
sm.flatten()

In [ ]:
wf_at_sm = sm(wf)

plt.figure(figsize=(18, 9))
plt.suptitle('SM in pupil plane - flattened')

plt.subplot(1, 2, 1)
imshow_field(wf_at_sm.intensity, mask=aper)
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(wf_at_sm.phase, mask=aper, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

The wavefront phase is displayed in units of radians.

In [ ]:
#for i in [2, 34, 3, 19, 23, 28, 14]:
for i in [19, 28]:
    sm.set_segment(i, 200e-9, 0, 0)   # This input is in meters, the phasemap below is in radians though.

In [ ]:
# Pupil plane SM image
wf_at_sm = sm(wf)

plt.figure(figsize=(18, 9))
plt.suptitle('SM in pupil plane - with pistons')

plt.subplot(1, 2, 1)
imshow_field(wf_at_sm.intensity, mask=aper)
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(wf_at_sm.phase, mask=aper, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

In [ ]:
# Image after SM with pistons
im = prop(sm(wf))

plt.figure(figsize=(18, 9))
plt.suptitle('Image plane after SM with pistons')

plt.subplot(1, 2, 1)
imshow_field(np.log10(im.intensity))
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(im.phase, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

In [ ]:
# Display piston aberrated and reference image next to each other
plt.figure(figsize=(18, 9))
plt.suptitle('Reference image and piston image')

plt.subplot(1, 2, 1)
imshow_field(np.log10(im_ref.intensity))
plt.title('Reference image')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(np.log10(im.intensity))
plt.title('Image with piston aberrations')
plt.colorbar()

In [ ]:
# Get the residual of the two images
res = im_ref.intensity - im.intensity
plt.figure(figsize=(15, 15))
imshow_field(res)
plt.title('Residual of image with pistons')
plt.colorbar()

## Comparison to Poppy HexDM with three rings

Lets first make a segmented mirror like ATLAST/HiCAT, as we did with HCIPy and calculate the unaberrated PSF.

In [ ]:
import poppy
import astropy.units as u
import matplotlib.pyplot as plt
%matplotlib inline

wvln_pop = 638e-9   # m

iris = poppy.dms.HexSegmentedDeformableMirror(name='Iris DM',
                                              rings=3,
                                              flattoflat=2.14*u.m,
                                              gap=2*u.cm,
                                              center=False)

In [ ]:
plt.figure(figsize=(16, 8))
iris.display(what='both')

In [ ]:
# Make an optical system with the segmented DM and a detector
osys = poppy.OpticalSystem()
osys.add_pupil(iris)
pxscle = 0.013      # I'm tweaking pixelscale and fov_arcsec to match the HCIPy image
fovarc = 0.28
osys.add_detector(pixelscale=pxscle, fov_arcsec=fovarc, oversample=8)

# Calculate the PSF
psf = osys.calc_psf(wvln_pop)
plt.figure(figsize=(15, 15))
poppy.display_psf(psf, vmin=1e-12, vmax=0.1)

# Get the PSF as an array
psf_poppy = psf[0].data

In [ ]:
# Display HCIPy and Poppy unaberrated images next to each other
# Mormalize them by their max and make sure we're using the same scale for both.
plt.figure(figsize=(20, 10))

plt.subplot(121)
imshow_field(np.log10(im_ref.intensity/np.max(im_ref.intensity)), vmin=-10, vmax=0)
plt.title('HCIPy unaberrated')
plt.colorbar()

plt.subplot(122)
plt.imshow(np.log10(psf_poppy/np.max(psf_poppy)), vmin=-10, vmax=0)
plt.title('Poppy unaberrated')
plt.colorbar()

### Aberrating two segments with piston

Careful, poppy has a different sugment numbering than HCIPy! Also, HCIPy and poppy both take the segment aberration input in meters, but the HCIPy displays convert them to radians while poppy keeps them in meters (of surface (!) aberration!).

In [ ]:
# Set some actuators
iris.flatten()

aber = 200*u.nm
print('Aberration: {}'.format(aber))
print('Aberration:  {} rad'.format(2*np.pi*aber.to(u.m).value/wvln_pop))

iris.set_actuator(34, aber, 0, 0)   # 34 in poppy is 19 in HCIPy
iris.set_actuator(25, aber, 0, 0)

# iris.set_actuator(15, aber, 0, 0)
# iris.set_actuator(12, aber, 0, 0)
# iris.set_actuator(9, aber, 0, 0)
# iris.set_actuator(30, aber, 0, 0)
# iris.set_actuator(16, aber, 0, 0)


plt.figure(figsize=(10,10))
iris.display(what='opd')

In [ ]:
# Create new optical system with aberrated SM
osys_aber = poppy.OpticalSystem()
osys_aber.add_pupil(iris)
osys_aber.add_detector(pixelscale=pxscle, fov_arcsec=fovarc, oversample=8)

# Calculate the PSF
psf = osys_aber.calc_psf(2e-6)
plt.figure(figsize=(15, 15))
poppy.display_psf(psf, vmin=1e-9, vmax=0.1)

# Get the PSF as an array
psf_poppy_aber = psf[0].data

It makes absolitely no sense that this image has a smaller fov than the previous Poppy image, since I use exactly the same setup. But I gave up on fixing it since I got a better comparison between Poppy and HCIPy results further down where I unify the aberration input between the two.

In [ ]:
plt.figure(figsize=(20, 10))

plt.subplot(121)
imshow_field(np.log10(im.intensity))
plt.title('HCIPy with piston aberrations')
plt.colorbar()

plt.subplot(122)
plt.imshow(np.log10(psf_poppy_aber))
plt.title('Aberrated poppy')
plt.colorbar()

These actualy match pretty well. I should try comparing a bigger field of view and also make sure the aberrations I put on are the same. For this I should figure out how to include a real wavelength into HCIPy, which might require also setting a real telescope size instead of a normalized aperture and a pupil grid of size 1. - I did this in the section below and in notebook 3c.

I also think that when I put 0.5 * wvnl_hc or more of aberration on HCIPy, I get phase wrapping, because all the aberrations in my image plane go away.

### Unify aberration input between HCIPy and Poppy

I want to be able to define a single aberration number that I then put on the SMs as a segment aberration. I can do that by defining the aberration in terms of radians of phase and then convert them to linear units, based on the wavelength used, at the input to the SM.

In [ ]:
# Make a new Poppy SM
poppysm = poppy.dms.HexSegmentedDeformableMirror(name='Poppy SM',
                                                 rings=3,
                                                 flattoflat=2.14*u.m,
                                                 gap=2*u.cm,
                                                 center=False)

In [ ]:
print('wvln_hc: {} m'.format(wvln_hc))
print('wvln_pop: {} m'.format(wvln_pop))

In [ ]:
# Define aberration in radians of phase and convert to OPD error in both systems
aber_rad = 1.2

aber_hc = aber_rad * wvln_hc / (2 * np.pi)
aber_pop = aber_rad * wvln_pop / (2 * np.pi)

print('Aberration: {} rad'.format(aber_rad))
print('aber_hc: {} m OPD(!)'.format(aber_hc))
print('aber_pop: {} m OPD(!)'.format(aber_pop))

In [ ]:
# Make a function out of this
def phase_to_linear(aber, wvln):
    """ Convert radians of phase aberration to linear aberration based on wavelength.
    -----------
    Parameters:
    aber: in radians
    wvln: wavelength in meters
    -------
    Reutrn:
    lin_aber: OPD aberration in meters
    """
    lin_aber = aber * wvln / (2 * np.pi)
    return lin_aber

In [ ]:
# Flatten both SMs
poppysm.flatten()
sm.flatten()
wf_at_sm = sm(wf)

# Plot HCIPy SM
plt.figure(figsize=(6,6))
imshow_field(wf_at_sm.phase, mask=aper, cmap='RdBu')
plt.title('HCIPy SM OPD')
plt.colorbar()

In [ ]:
# Plot Poppy SM
plt.figure(figsize=(7,7))
poppysm.display(what='opd')

In [ ]:
##########
# CASE 1 #
##########

# Put the aberration as piston on 4 segments on both SMs
poppysm.flatten()    # Flattening first in case I run this several times.
sm.flatten()

print('Phase aberration: {} rad'.format(aber_rad))

# HCIPy SM
print('HCIPY aberration: {} m'.format(phase_to_linear(aber_rad, wvln_hc)))
for i in [19, 28, 23, 16]:
    sm.set_segment(i, phase_to_linear(aber_rad, wvln_hc), 0, 0)
    
# Poppy SM
print('Poppy aberration: {} m'.format(phase_to_linear(aber_rad, wvln_pop)))
for i in [34, 25, 21, 14]:
    # remember that the poppy method needs an astropy quantity
    # also remember to divide by two since poppy input is in surface error instead of opd
    poppysm.set_actuator(i, phase_to_linear(aber_rad, wvln_pop)*u.m, 0, 0)

In [ ]:
##########
# CASE 2 # skip this cell if you can to test case 1
##########

# Put the aberration as piston on 4 segments on both SMs
poppysm.flatten()    # Flattening first in case I run this several times.
sm.flatten()

print('Phase aberration: {} rad'.format(aber_rad))

# HCIPy SM
print('HCIPY aberration: {} m'.format(phase_to_linear(aber_rad, wvln_hc)))
for i in [19, 28, 23, 16, 3, 35, 30, 8]:
    sm.set_segment(i, phase_to_linear(aber_rad, wvln_hc), 0, 0)
    
# Poppy SM
print('Poppy aberration: {} m'.format(phase_to_linear(aber_rad, wvln_pop)))
for i in [34, 25, 21, 14, 1, 32, 27, 10]:
    # remember that the poppy method needs an astropy quantity
    # also remember to divide by two since poppy input is in surface error instead of opd
    poppysm.set_actuator(i, phase_to_linear(aber_rad, wvln_pop)*u.m, 0, 0)

In [ ]:
# Plot HCIPy SM
wf_at_sm = sm(wf)

plt.figure(figsize=(6,6))
imshow_field(wf_at_sm.phase, mask=aper, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

In [ ]:
# Plot Poppy SM
plt.figure(figsize=(7,7))
poppysm.display(what='opd')

In [ ]:
# Propagte both SMs to image

# HCIPy
im_hc = prop(sm(wf))

# Poppy
osys_op = poppy.OpticalSystem()
osys_op.add_pupil(poppysm)
osys_op.add_detector(pixelscale=pxscle, fov_arcsec=fovarc, oversample=8)

psf = osys_op.calc_psf(wvln_pop)
im_poppy = psf[0].data

# Plot both images
plt.figure(figsize=(20, 10))

plt.subplot(121)
imshow_field(np.log10(im_hc.intensity))
plt.title('HCIPy')

plt.subplot(122)
plt.imshow(np.log10(im_poppy))
plt.title('Poppy')